## Import Libaries

In [3]:
# import all necessary libraries
import os
import datetime
from utils.Copernicus import AdvancedCopernicus
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import folium
from tqdm import tqdm
from pathlib import Path
import math

from datetime import datetime
# Ignore SettingWithCopyWarning:
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)



# Display all columns
pd.options.display.max_columns = None
#pd.options.display.max_rows = None

## Download Training Data

In [4]:
START_DATE = '2022-12-03'
END_DATE = '2025-03-22'
OUTPUT_FILENAME = 'output.nc'

BBOX= {
    
    # "minimum_longitude":9.10,
    # "maximum_longitude":30.20,
    # "minimum_latitude":53.10,
    # "maximum_latitude":65.00,

    "minimum_longitude":9.1,
    "maximum_longitude":23.0,
    "minimum_latitude":53.5,
    "maximum_latitude":60.0,

    # "minimum_longitude":9.85083510071235,
    # "maximum_longitude":10.926709174713364,
    # "minimum_latitude":54.25206332481298,
    # "maximum_latitude":54.97306793985031,

    "target_min_lon":10.156,
    "target_max_lon":10.170,
    "target_min_lat":54.354,
    "target_max_lat":54.365
    }

In [5]:
# Download Data
minimum_latitude = BBOX['minimum_latitude']
maximum_latitude = BBOX['maximum_latitude']
minimum_longitude = BBOX['minimum_longitude']
maximum_longitude = BBOX['maximum_longitude']

copernicus = AdvancedCopernicus()
points = 30
latitudes = np.linspace(minimum_latitude, maximum_latitude, points)
longitudes = np.linspace(minimum_longitude, maximum_longitude, points)



# zip latitudes and longitudes
coordinates = set([(float(lat), float(lon)) for lat in latitudes for lon in longitudes])
print(len(coordinates))

mean_lat = np.mean(latitudes)
mean_lon = np.mean(longitudes)
# plot the coordinates to folium
folium_map = folium.Map(location=[mean_lat, mean_lon], zoom_start=5)
folium.Polygon(locations=[[minimum_latitude, minimum_longitude], [minimum_latitude, maximum_longitude], [maximum_latitude, maximum_longitude], [maximum_latitude, minimum_longitude], [minimum_latitude, minimum_longitude]], color='orange').add_to(folium_map)
for lat, lon in coordinates:
    # plot the coordinates with dots
    

    #folium.Circle(location=[lat, lon], radius=30, color='red').add_to(folium_map)
    folium.CircleMarker(location=[lat, lon], radius=4, color='red').add_to(folium_map)
    #folium.Marker(location=[lat, lon], popup=f'lat: {lat}, lon: {lon}').add_to(folium_map)
folium_map

900


### Ocean Data

In [6]:
numerical_data_path = Path(f'../data/numerical_data/points{points}')
if not os.path.exists(numerical_data_path):
    os.makedirs(numerical_data_path)

In [7]:


# Daten die mit .nc enden
nc_files = [file for file in os.listdir(os.getcwd()) if file.endswith('.nc')]
nc_files = sorted(nc_files)


    
    
    


# Downlaod the .nc Data

variables = ["bottomT", "mlotst", "siconc", "sithick", "sla", "so", "sob", "thetao", "uo", "vo", "wo"]

for lat, lon in tqdm(coordinates, total=len(coordinates)):
    #print(lat, lon)

    file = f'cmems_mod_bal_phy_anfc_PT1H-i_{START_DATE}-{END_DATE}_lat{round(lat,3)}-lon{round(lon,3)}_points{points}.nc'
    
    # check if the file already exists in the data folder
    if file in os.listdir(numerical_data_path):
        print(f'{file} already exists')
        continue

    radius = 0.01 
    if lon-radius < BBOX['minimum_longitude'] or lon+radius > BBOX['maximum_longitude'] or lat-radius < BBOX['minimum_latitude'] or lat+radius > BBOX['maximum_latitude']:
        radius = 0




    ds = copernicus.get_subset(
        dataset_id="cmems_mod_bal_phy_anfc_PT1H-i",
        #dataset_version="202411",
        variables=variables,
        minimum_longitude=lon-radius,
        maximum_longitude=lon+radius,
        minimum_latitude=lat-radius,
        maximum_latitude=lat+radius,

        start_datetime=START_DATE,
        end_datetime=END_DATE,
        minimum_depth=0.5016462206840515,
        maximum_depth=0.5016462206840515,
        coordinates_selection_method="strict-inside",
        disable_progress_bar=True,
        output_filename=file,
        delete_file=False)
    # check if all clumns are NaN
    # if so, delete the file
    df = ds.to_dataframe()

    if df.isnull().all().all():
        
        os.remove(file)
        print(f'{file} is removed')
        continue
    

    # move the file to the data folder
    os.replace(file, Path(numerical_data_path, file))
        




  0%|          | 0/900 [00:00<?, ?it/s]

INFO - 2025-03-28T16:38:02Z - Selected dataset version: "202411"
INFO - 2025-03-28T16:38:02Z - Selected dataset part: "default"
INFO - 2025-03-28T16:38:10Z - Starting download. Please wait...
INFO - 2025-03-28T16:38:12Z - Successfully downloaded to cmems_mod_bal_phy_anfc_PT1H-i_2022-12-03-2025-03-22_lat54.845-lon10.059_points30.nc
  0%|          | 1/900 [00:10<2:36:49, 10.47s/it]INFO - 2025-03-28T16:38:12Z - Selected dataset version: "202411"
INFO - 2025-03-28T16:38:12Z - Selected dataset part: "default"
INFO - 2025-03-28T16:38:16Z - Starting download. Please wait...
INFO - 2025-03-28T16:38:16Z - Successfully downloaded to cmems_mod_bal_phy_anfc_PT1H-i_2022-12-03-2025-03-22_lat57.534-lon23.0_points30.nc
  0%|          | 2/900 [00:15<1:47:42,  7.20s/it]INFO - 2025-03-28T16:38:17Z - Selected dataset version: "202411"
INFO - 2025-03-28T16:38:17Z - Selected dataset part: "default"
INFO - 2025-03-28T16:38:20Z - Starting download. Please wait...
INFO - 2025-03-28T16:38:21Z - Successfully dow

cmems_mod_bal_phy_anfc_PT1H-i_2022-12-03-2025-03-22_lat54.172-lon21.562_points30.nc is removed


INFO - 2025-03-28T16:38:31Z - Selected dataset version: "202411"
INFO - 2025-03-28T16:38:31Z - Selected dataset part: "default"
INFO - 2025-03-28T16:38:35Z - Starting download. Please wait...
INFO - 2025-03-28T16:38:35Z - Successfully downloaded to cmems_mod_bal_phy_anfc_PT1H-i_2022-12-03-2025-03-22_lat55.293-lon21.562_points30.nc
  1%|          | 6/900 [00:33<1:14:03,  4.97s/it]

cmems_mod_bal_phy_anfc_PT1H-i_2022-12-03-2025-03-22_lat55.293-lon21.562_points30.nc is removed


...

INFO - 2025-03-28T17:52:54Z - Selected dataset version: "202411"
INFO - 2025-03-28T17:52:54Z - Selected dataset part: "default"
INFO - 2025-03-28T17:52:57Z - Starting download. Please wait...
INFO - 2025-03-28T17:52:58Z - Successfully downloaded to cmems_mod_bal_phy_anfc_PT1H-i_2022-12-03-2025-03-22_lat57.759-lon12.934_points30.nc
100%|█████████▉| 897/900 [1:14:56<00:14,  4.96s/it]

cmems_mod_bal_phy_anfc_PT1H-i_2022-12-03-2025-03-22_lat57.759-lon12.934_points30.nc is removed


INFO - 2025-03-28T17:52:58Z - Selected dataset version: "202411"
INFO - 2025-03-28T17:52:58Z - Selected dataset part: "default"
INFO - 2025-03-28T17:53:02Z - Starting download. Please wait...
INFO - 2025-03-28T17:53:03Z - Successfully downloaded to cmems_mod_bal_phy_anfc_PT1H-i_2022-12-03-2025-03-22_lat59.776-lon18.686_points30.nc
100%|█████████▉| 898/900 [1:15:01<00:10,  5.02s/it]

cmems_mod_bal_phy_anfc_PT1H-i_2022-12-03-2025-03-22_lat59.776-lon18.686_points30.nc is removed


INFO - 2025-03-28T17:53:04Z - Selected dataset version: "202411"
INFO - 2025-03-28T17:53:04Z - Selected dataset part: "default"
INFO - 2025-03-28T17:53:07Z - Starting download. Please wait...
INFO - 2025-03-28T17:53:08Z - Successfully downloaded to cmems_mod_bal_phy_anfc_PT1H-i_2022-12-03-2025-03-22_lat54.172-lon11.976_points30.nc
100%|█████████▉| 899/900 [1:15:06<00:05,  5.03s/it]INFO - 2025-03-28T17:53:09Z - Selected dataset version: "202411"
INFO - 2025-03-28T17:53:09Z - Selected dataset part: "default"
INFO - 2025-03-28T17:53:12Z - Starting download. Please wait...
INFO - 2025-03-28T17:53:12Z - Successfully downloaded to cmems_mod_bal_phy_anfc_PT1H-i_2022-12-03-2025-03-22_lat55.966-lon21.083_points30.nc
100%|██████████| 900/900 [1:15:11<00:00,  5.01s/it]
